In [1]:
import numpy as np
import pandas as pd

from astropy.io import fits

import auxil_ML as aux


In [43]:
cat = '3FGL'
if cat == '4FGL':
    #cat_name = '4FGL'
    cat_name = '4FGL-DR2'
else:
    cat_name = cat
    
folder = '../ML_3FGL_paper/SOM'
#folder = '../ML_3FGL_paper_extra/data/python files/catas'
cat_fn = '%s/%s_prob_catalog.csv' % (folder, cat_name)
cat_df = pd.read_csv(cat_fn, index_col=0)
labels = ['AGN', 'PSR', 'MIXED']
res_dict = {}
res_dict['uncorrected'] = {}
res_dict['corrected'] = {}


In [44]:
if cat_name == '4FGL-DR2':
    fn = 'data/gll_psc_v27.fit'
elif cat_name == '4FGL':
    fn = 'data/gll_psc_v22.fit'
elif cat_name == '3FGL':
    fn = '../PS_comparison/gll_psc_v16.fit'
    
with fits.open(fn) as data:
    data[1].header
    cat0_df = aux.hdu2df(data[1], index_name='Source_Name')

print(len(cat0_df), len(cat_df))
names0 = list(cat0_df.index)
for name in cat_df.index:
    names0.remove(name)
print(len(names0))

3034 3021
13


In [45]:
cat0_df.columns

Index(['RAJ2000', 'DEJ2000', 'GLON', 'GLAT', 'Conf_68_SemiMajor',
       'Conf_68_SemiMinor', 'Conf_68_PosAng', 'Conf_95_SemiMajor',
       'Conf_95_SemiMinor', 'Conf_95_PosAng', 'ROI_num', 'Signif_Avg',
       'Pivot_Energy', 'Flux_Density', 'Unc_Flux_Density', 'Flux1000',
       'Unc_Flux1000', 'Energy_Flux100', 'Unc_Energy_Flux100', 'Signif_Curve',
       'SpectrumType', 'Spectral_Index', 'Unc_Spectral_Index', 'beta',
       'Unc_beta', 'Cutoff', 'Unc_Cutoff', 'Exp_Index', 'Unc_Exp_Index',
       'PowerLaw_Index', 'Flux30_100', 'Unc_Flux30_100', 'nuFnu30_100',
       'Sqrt_TS30_100', 'Flux100_300', 'nuFnu100_300', 'Sqrt_TS100_300',
       'Flux300_1000', 'nuFnu300_1000', 'Sqrt_TS300_1000', 'Flux1000_3000',
       'nuFnu1000_3000', 'Sqrt_TS1000_3000', 'Flux3000_10000',
       'nuFnu3000_10000', 'Sqrt_TS3000_10000', 'Flux10000_100000',
       'nuFnu10000_100000', 'Sqrt_TS10000_100000', 'Variability_Index',
       'Signif_Peak', 'Flux_Peak', 'Unc_Flux_Peak', 'Time_Peak',
       'Peak_I

In [46]:
if 0:
    fn = '../ML_3FGL_paper_extra/data/python files/catas/%s_missing_values_src_test.csv' % cat_name
    print(fn)
    miss_df.to_csv(fn, index_label='%s_Source_Name' % cat)

In [47]:
unass_mask = cat_df['Category_%s' % cat] == 'UNAS'
ass_mask = cat_df['Category_%s' % cat] != 'UNAS'
other_mask = cat_df['Category_%s' % cat] == 'OTHER'
n_unas = np.sum(unass_mask)
n_as = np.sum(ass_mask)
print(n_as, n_unas)

2013 1008


In [48]:
set(cat_df['Category_%s' % cat])

{'AGN', 'OTHER', 'PSR', 'UNAS'}

In [49]:
np.sum(cat_df['Category_%s' % cat] == 'PSR')

166

In [50]:
for label in labels:
    res_dict['uncorrected'][label] = np.sum((cat_df['Category_Prob'] == label) & unass_mask)
    print('%s: %i' % (label, res_dict['uncorrected'][label]))
print('Total:', n_unas)

AGN: 597
PSR: 111
MIXED: 300
Total: 1008


In [51]:
# outliers
for src in ['AGN', 'PSR']:
    total = 1. * sum((cat_df['Category_%s' % cat] == src))
    nmixed = sum((cat_df['Category_%s' % cat] == src) & (cat_df['Category_Prob'] == 'MIXED'))
    nwrong = sum((cat_df['Category_%s' % cat] == src) & (cat_df['Category_Prob'] != src)) - nmixed
    print('%s %s (%i total)' %(cat, src, total))
    print('Mixed: %i (%.3f)' % (nmixed, nmixed/total))
    print('Wrong: %i (%.3f)' % (nwrong, nwrong/total))

3FGL AGN (1739 total)
Mixed: 108 (0.062)
Wrong: 3 (0.002)
3FGL PSR (166 total)
Mixed: 46 (0.277)
Wrong: 8 (0.048)


In [52]:
sum((cat_df['Category_%s' % cat] == 'PSR') & (cat_df['Category_Prob'] == 'AGN'))

8

In [53]:
print('4FGL-DR2 number of sources:', 5064 + 723)
print(len(cat_df))
print(len(cat0_df))

4FGL-DR2 number of sources: 5787
3021
3034


In [54]:
for label in labels:
    n_other = np.sum((cat_df['Category_Prob'] == label) & other_mask)
    n_label = 1. * np.sum((cat_df['Category_Prob'] == label) & ass_mask)
    print('%s other: %i' % (label, n_other))
    if n_label:
        print('%s other / n assoc: %.2f' % (label, n_other / n_label))
    n_corr = 1. * n_other * n_unas / n_as
    res_dict['corrected'][label] = res_dict['uncorrected'][label] - n_corr
    print('%s uncorrected: %i' % (label, res_dict['uncorrected'][label]))
    print('%s corrected: %.1f' % (label, res_dict['corrected'][label]))
print('Total other:', sum(other_mask))

AGN other: 38
AGN other / n assoc: 0.02
AGN uncorrected: 597
AGN corrected: 578.0
PSR other: 28
PSR other / n assoc: 0.20
PSR uncorrected: 111
PSR corrected: 97.0
MIXED other: 42
MIXED other / n assoc: 0.21
MIXED uncorrected: 300
MIXED corrected: 279.0
Total other: 108
